# 1. Initializing Variables and Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
import sys
import csv
import time
import heapq
import tweepy # https://github.com/tweepy/tweepy
import numpy as np
import pandas as pd
import configparser
import tensorflow as tf

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, LSTM, Dropout, GRU, TimeDistributed, BatchNormalization
from keras.layers import CuDNNLSTM 
from keras.layers.core import Dense, Activation, Dropout, RepeatVector

from keras.utils import np_utils
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint


In [ ]:
# twitter handler of user to fetch data for
Handler = "elonmusk" #@param {type:"string"}


In [ ]:
# initializing variables
path    = 'drive/MyDrive/Colab Notebooks/COE494_Project'
Model   =  {
      'Type': 'LSTM',
      'seed' : {}
    }

# 2. Importing and Grabbing the data

In [ ]:
# Twitter API authentication 
def authenticate(path = 'drive/MyDrive/Colab Notebooks/COE494_Project/'):
  # read config
  config = configparser.ConfigParser()
  config.read(path + 'config.ini')

  api_key = str(config['twitter']['api_key'])
  api_key_secret = str(config['twitter']['api_key_secret'])

  access_token = str(config['twitter']['access_token'])
  access_token_secret = str(config['twitter']['access_token_secret'])

  # authenticate
  auth = tweepy.OAuthHandler(api_key, api_key_secret)
  auth.set_access_token(access_token, access_token_secret)

  return tweepy.API(auth, wait_on_rate_limit = True)

In [ ]:
api = authenticate()

In [ ]:
# Function to fetch latest user tweets
def get_all_tweets(handler):
    # Twitter only allows access to a users most recent 3240 tweets with this method
    print(f'Grabbing @{handler}\'s Tweets')
    #initialize a list to hold all the tweepy Tweets
    all_tweets = []  
    
    # make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = handler, count = 200, include_rts = False, tweet_mode = 'extended')
    
    # save most recent tweets
    all_tweets.extend(new_tweets)
    
    # save the id of the oldest tweet less one
    oldest = all_tweets[-1].id - 1
    
    # keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:        
        # all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = handler, count=200, max_id = oldest, include_rts = False, tweet_mode = 'extended')
        # save most recent tweets
        all_tweets.extend(new_tweets)        
        # update the id of the oldest tweet less one
        oldest = all_tweets[-1].id - 1
        
    print(f"{len(all_tweets)} tweets downloaded...")    
    # transform the tweepy tweets into a 2D array that will populate the csv 
    out_tweets = [[tweet.id_str, tweet.created_at, tweet.full_text] for tweet in all_tweets]
    df = pd.DataFrame (out_tweets, columns = ["id", "time", "tweet"])
    df.to_csv(path + '/data/' + handler+'.csv')
    return df

In [ ]:
# Fetching, reading of tweets
# tweets = get_all_tweets(Handler)
tweets = pd.read_csv(path + "/data/" + Handler + ".csv")
print('Tweets Fetched -')
tweets.head()

Tweets Fetched -


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,4,9,14,16,17,19,22,24,25,26,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1.543473e+10,1.543473e+10,1280000000000.0,04/06/2010 18:31,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1.520000e+17,1.520000e+17,1330000000000.0,28/12/2011 22:27,0,NaN,@TheOnion So true :),en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1.520000e+17,1.520000e+17,1330000000000.0,27/12/2011 23:38,0,NaN,If you ever wanted to know the *real* truth ab...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1.510000e+17,1.510000e+17,1320000000000.0,26/12/2011 16:29,0,NaN,Walked around a neighborhood recently rebuilt ...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1.510000e+17,1.510000e+17,1320000000000.0,26/12/2011 16:23,0,NaN,"It was Xmas, so we brought presents for the ki...",en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


# 3. Cleaning the data

In [ ]:
# Tweet text pre-processing
def clean_tweet(tweet):
    stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from", "amp"]
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp

In [ ]:
# cleaning tweets
cleaned_tweets = pd.DataFrame([clean_tweet(tweet) for tweet in tweets.tweet], columns = ['tweet'])

# Removing null and empty rows after cleaning
cleaned_tweets.tweet.replace('', np.nan, inplace=True)
cleaned_tweets.dropna(inplace = True)

print('Tweets Cleaned -')
cleaned_tweets.head()

Tweets Cleaned -


,tweet
0,please ignore prior tweets as that was someone...
1,so true
2,if you ever wanted know real truth about moon ...
3,walked around neighborhood recently rebuilt wi...
4,it was xmas so we brought presents kids at orp...


# 4. Feature Engineering

Flattening the cleaned tweets data from a dataframe to a string

In [ ]:
raw_tweets_text = ' '.join(cleaned_tweets["tweet"])

In [ ]:
# Tokenization, segmentation and text processing functions 
def prepare_input(text):
    x = np.zeros((1, SEQUENCE_LENGTH, len(chars)))
    for t, char in enumerate(text):
        x[0, t, char_indices[char]] = 1.
    return x

def temperatureSample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char

        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion

def predict_completions(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_indices]


In [ ]:
text = raw_tweets_text
print('corpus length:', len(text))

# initializing vocab and translators
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(f'unique chars: {len(chars)}')

# initializing important parameters
SEQUENCE_LENGTH = 80
step = 4
sentences = []
next_chars = []

# segmenting the data
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])
print(f'num training examples: {len(sentences)}')

# Vectorization and initializing zero arrays for one hot encoding
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

# encoding the data
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print("X.shape:", X.shape)
print("y.shape:", y.shape)

corpus length: 2222261
unique chars: 37
num training examples: 555546
X.shape: (555546, 80, 37)
y.shape: (555546, 37)


# 5. Model Building

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model = Sequential()

model.add(CuDNNLSTM(len(chars) * 5, input_shape=(SEQUENCE_LENGTH, len(chars))))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars) * 2))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars) * 2))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr = 0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cu_dnnlstm (CuDNNLSTM)      (None, 185)               165760    
                                                                 
 batch_normalization (BatchN  (None, 185)              740       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 185)               0         
                                                                 
 dense (Dense)               (None, 74)                13764     
                                                                 
 batch_normalization_1 (Batc  (None, 74)               296       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 74)                0

In [ ]:
model.fit(X, y, validation_split = 0.05, batch_size = 124, epochs = 50, shuffle = True)

Epoch 1/50
4257/4257 [==============================] - 64s 13ms/step - loss: 2.0974 - accuracy: 0.3700 - val_loss: 2.1691 - val_accuracy: 0.3421
Epoch 2/50
4257/4257 [==============================] - 54s 13ms/step - loss: 1.8266 - accuracy: 0.4432 - val_loss: 1.8652 - val_accuracy: 0.4304
Epoch 3/50
4257/4257 [==============================] - 54s 13ms/step - loss: 1.6874 - accuracy: 0.4838 - val_loss: 1.7055 - val_accuracy: 0.4829
Epoch 4/50
4257/4257 [==============================] - 54s 13ms/step - loss: 1.5922 - accuracy: 0.5100 - val_loss: 1.6421 - val_accuracy: 0.4915
Epoch 5/50
4257/4257 [==============================] - 54s 13ms/step - loss: 1.5247 - accuracy: 0.5283 - val_loss: 1.6030 - val_accuracy: 0.5111
Epoch 6/50
4257/4257 [==============================] - 54s 13ms/step - loss: 1.4727 - accuracy: 0.5428 - val_loss: 1.5918 - val_accuracy: 0.5114
Epoch 7/50
4257/4257 [==============================] - 54s 13ms/step - loss: 1.4309 - accuracy: 0.5546 - val_loss: 1.5553 -

# 6. Tweet Prediction and Text Generator

In [ ]:
def genSentence(text, words = 2):
    textOG = text
    text = text.lower()
    while len(text) < SEQUENCE_LENGTH:
        text = ' ' + text
    text = text[-SEQUENCE_LENGTH:]
    for i in range(words):
        text = text[-SEQUENCE_LENGTH:]
        pred = predict_completions(text, 1)[0]
        text = text + pred
        textOG = textOG + pred
        pass
    return textOG

Output 1, using the first seed

In [ ]:
# Predict Tweets starting from the given words 
# Generates random length from the training set

# Random Number of words and seed word may be generated, however we are choosing
# constant values for comparison sake
N_WORDS = 0
while N_WORDS < 10 or N_WORDS > 50:
  N_WORDS = len(cleaned_tweets.sample().tweet.item())

seed_1 = cleaned_tweets.sample().tweet.item().split()[0]
Model['seed'][seed_1] = genSentence(seed_1, N_WORDS)

print('Prompt:', seed_1)
print('Prediction:', Model['seed'][seed_1])

Prompt: will
Prediction: will probably autocuthered that we week publication is that it s not that it s indeed eserted obv approvals performance people viever i superchargers come point i lived ever systemicb 


Output 2, using the second seed

In [ ]:
# Predict Tweets starting from the given words 
# Generates random length from the training set

# Random Number of words and seed word may be generated, however we are choosing
# constant values for comparison sake
N_WORDS = 0
while N_WORDS < 10 or N_WORDS > 50:
  N_WORDS = len(cleaned_tweets.sample().tweet.item())

seed_2 = cleaned_tweets.sample().tweet.item().split()[0]
Model['seed'][seed_2] = genSentence(seed_2, N_WORDS)

print('Prompt:', seed_2)
print('Prediction:', Model['seed'][seed_2])

Prompt: going
Prediction: going regularmeahopens should be super heavy so they will be super card that i do put this is that it s not that it s indeed eserted obv approvals performance people viever i superchargers come point i lived ever systemicb story that was easy 


# 7. Model evaluation

In [ ]:
from prettytable import PrettyTable
from nltk.translate import bleu_score as bleu

In [ ]:
model_gens  = [gen for gen in list(Model['seed'].values())]
# transforming the training data/referencing tweets into appropriate format
reference_tweets = [sentence.split() for sentence in cleaned_tweets["tweet"].tolist()]

In [ ]:
# seed scores
bleu_scors = [round(bleu.sentence_bleu(reference_tweets, tweet.split(), smoothing_function = bleu.SmoothingFunction().method4), 2) for tweet in model_gens]
bleu_avg   = sum(bleu_scors) / len(bleu_scors)

In [ ]:
table = ['seed', Model['Type']], [seed_1, bleu_scors[0]], [seed_2, bleu_scors[1]], ['average scores', bleu_avg]
tab = PrettyTable(table[0])
tab.add_rows(table[1:])
print(tab.get_string(title="BLUE Evaluation Scores"))

+--------------------------+
|  BLUE Evaluation Scores  |
+------------------+-------+
|       seed       |  LSTM |
+------------------+-------+
|       will       |  0.26 |
|      going       |  0.26 |
|  average scores  |  0.26 |
+------------------+-------+
